#### Download and explore dataset

In [2]:
import pandas as pd
# Read the CSV file
all_data = pd.read_csv("data/monatszahlen2311_verkehrsunfaelle_export_24_11_23_r.csv")

# View the first 5 rows
all_data.head()

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT,VORJAHRESWERT,VERAEND_VORMONAT_PROZENT,VERAEND_VORJAHRESMONAT_PROZENT,ZWOELF_MONATE_MITTELWERT
0,Alkoholunfälle,insgesamt,2023,202301,NaN,19.0,NaN,NaN,NaN
1,Alkoholunfälle,insgesamt,2023,202302,NaN,32.0,NaN,NaN,NaN
2,Alkoholunfälle,insgesamt,2023,202303,NaN,28.0,NaN,NaN,NaN
3,Alkoholunfälle,insgesamt,2023,202304,NaN,34.0,NaN,NaN,NaN
4,Alkoholunfälle,insgesamt,2023,202305,NaN,57.0,NaN,NaN,NaN


In [3]:
for col_name in all_data.columns:
    print(col_name + " options: ", all_data[col_name].unique()[:20])

MONATSZAHL options:  ['Alkoholunfälle' 'Fluchtunfälle' 'Verkehrsunfälle']
AUSPRAEGUNG options:  ['insgesamt' 'Verletzte und Getötete' 'mit Personenschäden']
JAHR options:  [2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010
 2009 2008 2007 2006 2005 2004]
MONAT options:  ['202301' '202302' '202303' '202304' '202305' '202306' '202307' '202308'
 '202309' '202310' '202311' '202312' 'Summe' '202201' '202202' '202203'
 '202204' '202205' '202206' '202207']
WERT options:  [ nan 493.  19.  32.  28.  34.  57.  60.  54.  44.  56.  49. 383.  16.
  14.  24.  48.  46.  43.  33.]
VORJAHRESWERT options:  [ 19.  32.  28.  34.  57.  60.  54.  44.  56.  49. 383.  16.  14.  24.
  48.  46.  43.  33.  21. 430.]
VERAEND_VORMONAT_PROZENT options:  [   nan  -9.52  68.42 -12.5   21.43  67.65   5.26 -10.   -18.52  27.27
 -42.86  14.29  23.08  71.43 -33.33  50.   100.    -8.33   4.55  17.39]
VERAEND_VORJAHRESMONAT_PROZENT options:  [   nan  28.72  18.75 128.57  16.67 112.5  137.5   25.    22.7